371024 Introducción al Aprendizaje Máquina  
Proyecto Tercer Parcial  
`Lic. Jose Ricardo Mendoza Villar`

# 1. Escoger una base de datos con al menos 5 columnas para entrenar un modelo de aprendizaje máquina. Si es posible, incluir el link a la base de datos.
- Base de Datos: https://www.kaggle.com/datasets/parulpandey/palmer-archipelago-antarctica-penguin-data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("penguins_size.csv")

In [3]:
df.head(3)

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE


In [4]:
# Las variables categóricas se convierten en su representación binarizada
dummies_columns = ["island", "sex"]
for dummie_column in dummies_columns:
    df_dummie = pd.get_dummies(df[dummie_column], drop_first = True)
    df = pd.concat([df_dummie, df], axis = 1)
    df = df.drop([dummie_column], axis = 1)

In [5]:
# Registros nulos
df.isna().sum()[df.isna().sum() > 0]

culmen_length_mm     2
culmen_depth_mm      2
flipper_length_mm    2
body_mass_g          2
dtype: int64

In [6]:
# Al ser minima la cantidad de registros, se eliminaran los datos con presencia de nulos en la base de datos
df = df.dropna()

In [7]:
df.head(3)

,FEMALE,MALE,Dream,Torgersen,species,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g
0,False,True,False,True,Adelie,39.1,18.7,181.0,3750.0
1,True,False,False,True,Adelie,39.5,17.4,186.0,3800.0
2,True,False,False,True,Adelie,40.3,18.0,195.0,3250.0


# 2. Escoger una de las columnas de la base de datos como variable objetivo (preferentemente la variable objetivo debe tomar más de dos valores). Las columnas restantes serán utilizadas como variables descriptivas.

In [8]:
X = df.drop("species", axis = 1)

In [9]:
y = df["species"]

# 3. Separar la base de datos en una de entrenamiento y otra de prueba.

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    stratify = y,
    random_state=20250517
)

# 4. Entrenar tres modelos de aprendizaje: Perceptrón, Bayes Ingenuo (¿qué versión es más adecuada?) y Máquina de Soporte Vectorial.

### Perceptrón

In [12]:
from sklearn.linear_model import Perceptron

In [13]:
perceptron = Perceptron(
    max_iter = 5000,
    eta0 = 0.05,
    random_state = 20250613
)

In [14]:
perceptron.fit(X_train, y_train)

Perceptron(eta0=0.05, max_iter=5000, random_state=20250613)

### Bayes Ingenuo  
Para este problema, es adecuado utilizar la versión `GaussianNB` debido a que el conjunto de datos incluyen variables continuas, asumiendo que estas se distribuyen de forma normal.

In [15]:
from sklearn.naive_bayes import GaussianNB

In [16]:
gnb = GaussianNB()

In [17]:
gnb.fit(X_train, y_train)

GaussianNB()

### Máquina de Soporte Vectorial

In [18]:
from sklearn.svm import SVC

In [19]:
svm_lineal = SVC(kernel = "linear")
svm_poly = SVC(kernel = "poly", degree = 4)
svm_sig = SVC(kernel = "sigmoid")
svm_rbf = SVC(kernel = "rbf")

In [20]:
svm_lineal.fit(X_train, y_train)
svm_poly.fit(X_train, y_train)
svm_sig.fit(X_train, y_train)
svm_rbf.fit(X_train, y_train)

SVC()

In [21]:
(
    svm_lineal.score(X_train, y_train),
    svm_poly.score(X_train, y_train),
    svm_sig.score(X_train, y_train),
    svm_rbf.score(X_train, y_train)
)

(0.99581589958159, 0.7280334728033473, 0.23430962343096234, 0.7322175732217573)

# 5. Obtener la exactitud (accuracy), matriz de confusión, reporte de clasificación de los tres modelos y compararlos.

In [22]:
from sklearn.metrics import classification_report, confusion_matrix

In [23]:
def prettyPrintMetrics(model_name: str, model, X_train, y_train, X_test, y_test) -> None: 
    print(f"{'-'*20} {model_name} {'-'*20}")
    print(f"Accuracy Train: {model.score(X_train, y_train)}")
    print(f"Accuracy Test: {model.score(X_test, y_test)}")
    print(confusion_matrix(y_test, model.predict(X_test)))
    print(classification_report(y_test, model.predict(X_test), zero_division = 0))

In [24]:
prettyPrintMetrics("Perceptrón", perceptron, X_train, y_train, X_test, y_test)
prettyPrintMetrics("Bayes Ingenuo", gnb, X_train, y_train, X_test, y_test)
prettyPrintMetrics("Máquina de Soporte Vectorial", svm_lineal, X_train, y_train, X_test, y_test)

-------------------- Perceptrón --------------------
Accuracy Train: 0.3598326359832636
Accuracy Test: 0.3592233009708738
[[ 0  0 45]
 [ 0  0 21]
 [ 0  0 37]]
              precision    recall  f1-score   support

      Adelie       0.00      0.00      0.00        45
   Chinstrap       0.00      0.00      0.00        21
      Gentoo       0.36      1.00      0.53        37

    accuracy                           0.36       103
   macro avg       0.12      0.33      0.18       103
weighted avg       0.13      0.36      0.19       103

-------------------- Bayes Ingenuo --------------------
Accuracy Train: 0.891213389121339
Accuracy Test: 0.8640776699029126
[[31 14  0]
 [ 0 21  0]
 [ 0  0 37]]
              precision    recall  f1-score   support

      Adelie       1.00      0.69      0.82        45
   Chinstrap       0.60      1.00      0.75        21
      Gentoo       1.00      1.00      1.00        37

    accuracy                           0.86       103
   macro avg       0.87    

El modelo de máquina de soporte vectorial fue el más efectivo, clasificando con total precisión los tres tipos de pingüinos para el conjunto de prueba y con un accuracy muy alto (prácticamente del 100%) para el conjunto de entrenamiento. El modelo de Bayes Ingenuo mostró un precisión muy buena para dos especies de pingüinos, el principal sesgo es que el modelo clasifica incorrectamente pingüinos de la especie Adelie como Chinstrap, para el modelo perceptrón no fue capaz de capturar las relaciones entre variables, posiblemente a la naturaleza lineal del modelo y la distribución de las clases en los modelos, este proyecto muestra claramente la importancia de probar distintos modelos al tener una tarea de aprendizaje máquina, puesto que cada conjunto de datos es único y puede o no cumplir con los supuestos que cada modelo necesita, aunque estos resuelvan la misma tarea (para este caso particular el problema de clasificación de múltiples características)